In [1]:
import numpy as np #matrix math 
import tensorflow as tf #machine learningt
import helpers #for formatting data into batches and generating random sequence data
from tensorflow.python.ops.rnn_cell import LSTMCell, LSTMStateTuple
from tensorflow.python.layers.core import Dense
from datasets.twitter import data
import data_utils
from tensorlayer.layers import *
import tensorlayer as tl
import time, sys
from chatbot_model import chatbot_seq2seq


In [2]:
import string

In [3]:
tf.reset_default_graph()

In [4]:
metadata, idx_q, idx_a = data.load_data(PATH='datasets/twitter/')
xvocab_size = len(metadata['idx2w'])  
# updata parameters with preprocessing
w2idx = metadata['w2idx']
idx2w = metadata['idx2w']
unk_id = w2idx['unk']
pad_id = w2idx['_']
start_id = xvocab_size
end_id = xvocab_size+1
w2idx.update({'start_id': start_id})
w2idx.update({'end_id': end_id})


In [5]:
emb_dim = 1024

In [6]:
idx2w = idx2w + ['start_id', 'end_id']
xvocab_size = yvocab_size = xvocab_size + 2

In [7]:
keys = w2idx.keys()

In [8]:
translator = str.maketrans('', '', string.punctuation)

In [9]:
def text_to_seq(text) :
    text = text.translate(translator)
    seq = []
    for word in text.lower().split():
        word_id = w2idx[word] if word in keys else 1
        seq.append(word_id)
    return seq

In [10]:
def seq_to_text(seq) :
    text = ''
    for word_id in seq :
        if word_id == end_id or word_id == pad_id :
            break;
        else :
            text += (idx2w[word_id]) + ' '
    return text

In [11]:
model = chatbot_seq2seq(
    vocab_size = xvocab_size, 
    input_embedding_size = emb_dim, 
    encoder_hidden_units = emb_dim, 
    decoder_hidden_units = 2 * emb_dim
)

In [12]:
def build_feeder(model, text, go_id, end_id):
    seq = text_to_seq(text)
    seq_len = len(seq)
    seq = np.reshape(np.array(seq), (seq_len,1))
    return {
        model.encoder_inputs : seq,
        model.go_tokens: np.zeros(1) + go_id,
        model.end_token: end_id
    }

In [13]:
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,device_count={'GPU':1}))

In [14]:
saver = tf.train.Saver()
checkpoint_prefix = 'ckpt_dir'
if tf.train.checkpoint_exists(checkpoint_prefix) :
    saver.restore(sess, tf.train.latest_checkpoint(checkpoint_prefix))
    print('Model successfully restored, chatbot ready to chat!')

INFO:tensorflow:Restoring parameters from ckpt_dir/model.ckpt
Model successfully restored, chatbot ready to chat!


In [25]:
fd = build_feeder(model, 'hello', start_id, end_id)

In [26]:
outs= sess.run(model.beam_outputs, fd)

In [27]:
outs.predicted_ids


array([[[  68, 3617, 7434, 1049, 2263]],

       [[  68, 3617, 7434, 1049, 2263]],

       [[  68, 3617, 7434, 1049, 2263]],

       [[  68, 3617, 7434, 1049, 2263]],

       [[  68, 3617, 7434, 1049, 2263]],

       [[  68, 3617, 7434, 1049, 2263]],

       [[  68, 3617, 7434, 1049, 2263]],

       [[  68, 3617, 7434, 1049, 2263]],

       [[  68, 3617, 7434, 1049, 2263]],

       [[  68, 3617, 7434, 1049, 2263]],

       [[  68, 3617, 7434, 1049, 2263]],

       [[  68, 3617, 7434, 1049, 2263]],

       [[  68, 3617, 7434, 1049, 2263]],

       [[  68, 3617, 7434, 1049, 2263]],

       [[  68, 3617, 7434, 1049, 2263]],

       [[  68, 3617, 7434, 1049, 2263]],

       [[  68, 3617, 7434, 1049, 2263]],

       [[  68, 3617, 7434, 1049, 2263]],

       [[  68, 3617, 7434, 1049, 2263]],

       [[  68, 3617, 7434, 1049, 2263]],

       [[  68, 3617, 7434, 1049, 2263]],

       [[  68, 3617, 7434, 1049, 2263]],

       [[  68, 3617, 7434, 1049, 2263]],

       [[  68, 3617, 7434, 1049, 2

In [24]:
idx2w[7068]

'quoting'

In [26]:
c.shape

TensorShape([Dimension(None), Dimension(4096)])

In [19]:
seq_to_text(np.reshape(out,out.shape[0]).tolist())

NameError: name 'out' is not defined